Set up env and imports

In [23]:
!pip install nada-dsl==0.5.0
!pip install py-nillion-client==0.5.0
!pip install nillion-python-helpers==0.2.3
!pip install python-dotenv==1.0.0


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import py_nillion_client as nillion
from py_nillion_client import NodeKey, UserKey
from dotenv import load_dotenv
from nillion_python_helpers import get_quote_and_pay, create_nillion_client, create_payments_config
from cosmpy.aerial.client import LedgerClient
from cosmpy.aerial.wallet import LocalWallet
from cosmpy.crypto.keypairs import PrivateKey

In [4]:
import os

load_dotenv('/Users/vishakh/.config/nillion/nillion-devnet.env')

for key, value in os.environ.items():
    print(f"{key}: {value}")

PATH: /Users/vishakh/dev/MonadicDNA/services/nillion-interactor/nillion-venv/bin:/Users/vishakh/.nvm/versions/node/v20.12.2/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/Library/Apple/usr/bin:/Users/vishakh/.cargo/bin:/Users/vishakh/.nilup/bin:/Users/vishakh/.foundry/bin
MANPATH: /Users/vishakh/.nvm/versions/node/v20.12.2/share/man:/opt/homebrew/share/man::
IJ_RESTARTER_LOG: /Users/vishakh/Library/Logs/JetBrains/IntelliJIdea2024.1/restarter.log
HOMEBREW_PREFIX: /opt/homebrew
LANG: en_US.UTF-8
COMMAND_MODE: unix2003
PS1: (nillion-venv) 
NVM_INC: /Users/vishakh/.nvm/versions/node/v20.12.2/include/node
LOGNAME: vishakh
HOMEBREW_REPOSITORY: /opt/homebrew
XPC_SERVICE_NAME: application.com.jetbrains.in

In [5]:
cluster_id = os.getenv('NILLION_CLUSTER_ID')
chain_id = os.getenv('NILLION_NILCHAIN_CHAIN_ID')
grpc_endpoint = os.getenv('NILLION_NILCHAIN_GRPC')

print(f"Cluster ID: {cluster_id}")
print(f"Chain ID: {chain_id}")
print(f"GRPC Endpoint: {grpc_endpoint}")

Cluster ID: 9e68173f-9c23-4acc-ba81-4f079b639964
Chain ID: nillion-chain-devnet
GRPC Endpoint: localhost:26649


In [6]:
node_seed = "my_seed"
nodekey = NodeKey.from_seed(node_seed)

In [7]:
monadic_seed = "monadic_seed"
monadic_userkey = UserKey.from_seed(monadic_seed)
monadic_client = create_nillion_client(monadic_userkey, nodekey)
monadic_party_id = monadic_client.party_id
monadic_user_id = monadic_client.user_id

In [8]:
snipper_seed = "snipper_seed"
snipper_userkey = UserKey.from_seed(snipper_seed)
snipper_client = create_nillion_client(snipper_userkey, nodekey)
snipper_party_id = snipper_client.party_id
snipper_user_id = snipper_client.user_id

In [9]:
seed = "my_seed"
userkey = UserKey.from_seed(seed)
client = create_nillion_client(userkey, nodekey)
party_id = client.party_id
user_id = client.user_id

In [10]:
payments_config = create_payments_config(chain_id, grpc_endpoint)
payments_client = LedgerClient(payments_config)
payments_wallet = LocalWallet(
    PrivateKey(bytes.fromhex(os.getenv("NILLION_NILCHAIN_PRIVATE_KEY_0"))),
    prefix="nillion",
)

In [11]:
program_name = "double"
program_mir_path = f"binaries/double.nada.bin"

In [14]:
async def store_program():
    receipt_store_program = await get_quote_and_pay(
        client,
        nillion.Operation.store_program(program_mir_path),
        payments_wallet,
        payments_client,
        cluster_id,
    )

    # Store the program
    action_id = await client.store_program(
        cluster_id, program_name, program_mir_path, receipt_store_program
    )
    return action_id

# Use this in your Jupyter notebook cell
action_id = await store_program()
print(f"Action ID: {action_id}")

Getting quote for operation...


RuntimeError: requesting price quote: price quote query failed: price quote query failed: timed out

In [52]:
program_id = f"{user_id}/{program_name}"

new_secret = nillion.NadaValues(
    {
        "foo": nillion.SecretInteger(2),
    }
)

party_name = "Party1"

# Set permissions for the client to compute on the program
permissions = nillion.Permissions.default_for_user(client.user_id)
permissions.add_compute_permissions({client.user_id: {program_id}})

# Pay for and store the secret in the network and print the returned store_id
receipt_store = await get_quote_and_pay(
    client,
    nillion.Operation.store_values(new_secret, ttl_days=5),
    payments_wallet,
    payments_client,
    cluster_id,
)
# Store a secret
store_id = await client.store_values(
    cluster_id, new_secret, permissions, receipt_store
)

Getting quote for operation...


RuntimeError: requesting price quote: price quote query failed: price quote query failed: timed out

In [28]:
compute_bindings = nillion.ProgramBindings(program_id)
compute_bindings.add_input_party(party_name, party_id)
compute_bindings.add_output_party(party_name, party_id)

computation_time_secrets = nillion.NadaValues({})

# Pay for the compute
receipt_compute = await get_quote_and_pay(
    client,
    nillion.Operation.compute(program_id, computation_time_secrets),
    payments_wallet,
    payments_client,
    cluster_id,
)

# Compute on the secret
compute_id = await client.compute(
    cluster_id,
    compute_bindings,
    [store_id],
    computation_time_secrets,
    receipt_compute,
)

# 8. Return the computation result
print(f"The computation was sent to the network. compute_id: {compute_id}")
while True:
    compute_event = await client.next_compute_event()
    if isinstance(compute_event, nillion.ComputeFinishedEvent):
        print(f"✅  Compute complete for compute_id {compute_event.uuid}")
        print(f"🖥️  The result is {compute_event.result.value}")
        break

Getting quote for operation...
Quote cost is 3 unil
Submitting payment receipt 3 unil, tx hash 7E5592AFBD8F780BF54C0D50B26D7CBE8DAE04A09F49CD04638EF4C19D40ED8A
The computation was sent to the network. compute_id: 3662f8e9-1e86-4ca8-be8c-ce3de6ecbf9a
✅  Compute complete for compute_id 3662f8e9-1e86-4ca8-be8c-ce3de6ecbf9a
🖥️  The result is {'my_output': 4}
